In [1]:
import torch
import contractions
import unidecode
import re
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import load_dataset
import pandas as pd


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = T5Tokenizer.from_pretrained('castorini/doc2query-t5-base-msmarco')
Queriesmodel = T5ForConditionalGeneration.from_pretrained('castorini/doc2query-t5-base-msmarco')
Queriesmodel.to(device)

2023-05-13 00:31:22.848511: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-13 00:31:23.424083: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-12.0/lib64:/usr/local/cuda-12.0/lib64
2023-05-13 00:31:23.424131: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-12.0/lib64:/usr/local/cuda-12.0/lib64
2023-05-13 00:31:23.

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [2]:
df_docs=pd.read_csv('multinews_combined.csv')
df_docs.head()

,document,summary
0,The number of Americans seeking unemployment a...,– The unemployment rate dropped to 8.2% last m...
1,Shelly Sterling still wants to retain partial ...,"– Shelly Sterling plans ""eventually"" to divorc..."
2,"GAITHERSBURG, Md. (AP) — A small, private jet ...",– A twin-engine Embraer jet that the FAA descr...
3,"On Monday night, while the rest of the world w...",– Tucker Carlson is in deep doodoo with conser...
4,Image caption A man has admitted removing anot...,– What are the three most horrifying words in ...


In [3]:
def preprocess(doc):
    doc = unidecode.unidecode(doc) # transliterates any unicode string into the closest possible representation in ascii text.
    doc = contractions.fix(doc) # expands contractions                   
    doc = re.sub('[\t\n]', ' ', doc) # remove newlines and tabs
    doc = re.sub(r'@[A-Za-z0-9_]+', '', doc) # remove mentions
    doc = re.sub(r'#[A-Za-z0-9_]+', '', doc) #remove hashtags
    doc = re.sub(r'https?://[^ ]+', '', doc)
    doc = re.sub(r'www.[^ ]+', '', doc)
    doc = re.sub('[^A-Za-z]+', ' ', doc) # remove all characters other than alphabet
    doc = re.sub(' +', ' ', doc) # substitute any number of space with one space only
    doc = doc.strip() # remove spaces from begining and end and lower the text
    return doc

In [4]:
df_docs.shape

(56216, 2)

In [5]:
df_docs.dropna(inplace=True)

In [6]:
df_docs= df_docs[df_docs['document']!=""]
df_docs= df_docs[df_docs['summary']!=""]

In [7]:
df_docs = df_docs.drop_duplicates(subset=['document'])
df_docs= df_docs.reset_index()
df_docs['Id'] = df_docs.index
df_docs.head()

,index,document,summary,Id
0,0,The number of Americans seeking unemployment a...,– The unemployment rate dropped to 8.2% last m...,0
1,1,Shelly Sterling still wants to retain partial ...,"– Shelly Sterling plans ""eventually"" to divorc...",1
2,2,"GAITHERSBURG, Md. (AP) — A small, private jet ...",– A twin-engine Embraer jet that the FAA descr...,2
3,3,"On Monday night, while the rest of the world w...",– Tucker Carlson is in deep doodoo with conser...,3
4,4,Image caption A man has admitted removing anot...,– What are the three most horrifying words in ...,4


In [13]:
df_docs = df_docs.drop('index', axis=1)
df_docs.head()

,document,summary,Id
0,The number of Americans seeking unemployment a...,– The unemployment rate dropped to 8.2% last m...,0
1,Shelly Sterling still wants to retain partial ...,"– Shelly Sterling plans ""eventually"" to divorc...",1
2,"GAITHERSBURG, Md. (AP) — A small, private jet ...",– A twin-engine Embraer jet that the FAA descr...,2
3,"On Monday night, while the rest of the world w...",– Tucker Carlson is in deep doodoo with conser...,3
4,Image caption A man has admitted removing anot...,– What are the three most horrifying words in ...,4


In [14]:
df_docs.to_csv('multinews_combined.csv')

In [15]:
from sentence_transformers import  SentenceTransformer
corpus= df_docs['summary'].tolist()
model = SentenceTransformer('sentence-transformers/msmarco-MiniLM-L6-cos-v5')
corpusVectors= model.encode(corpus)

In [18]:
corpusVectors.shape

(55876, 384)

In [17]:
import h5py
with h5py.File('Multinews.hdf5', 'w') as f:
    f.create_dataset('train', data=corpusVectors)
    f.attrs['distance']= 'angular'

### News vs. Queries

In [27]:
in__= df_docs['description'].tolist()

In [30]:
df_docs['description'].isnull().sum()

0

In [28]:
def get_query(doc):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  input_ids = tokenizer.encode(doc, return_tensors='pt').to(device)
  my_set = set()
  outputs = Queriesmodel.generate(
      input_ids=input_ids,
      max_length=2000,
      do_sample=True,
      top_k=10,
      num_return_sequences=20)

  for i in range(3):
      my_set.add(tokenizer.decode(outputs[i], skip_special_tokens=True))
  return ". ".join(my_set)

In [ ]:
import csv
with open('AG_News_Queries.csv', mode='w', newline='', encoding= 'utf-8') as file:
    writer= csv.writer(file)
    writer.writerow(['Document', 'Query'])
    
    for doc in in__:
        row= (doc, get_query(doc))
        writer.writerow(row)
        torch.cuda.empty_cache()

In [16]:
df_queries= pd.read_csv('AG_News_Queries.csv')
df_queries.head()

,Document,Query
0,Reuters Short sellers Wall Street s dwindling ...,who are the cynics on reuters. what is the gre...
1,Reuters Private investment firm Carlyle Group ...,who is carlyle. who is carlyle group. carlyle ...
2,Reuters Soaring crude prices plus worries abou...,does a rising oil price affect market share. w...
3,Reuters Authorities have halted oil export flo...,why is pipeline shut in southern iraq. who has...
4,AFP Tearaway world oil prices toppling records...,what are the oil prices?. what is the oil price


In [17]:
df_queries.drop_duplicates()

,Document,Query
0,Reuters Short sellers Wall Street s dwindling ...,who are the cynics on reuters. what is the gre...
1,Reuters Private investment firm Carlyle Group ...,who is carlyle. who is carlyle group. carlyle ...
2,Reuters Soaring crude prices plus worries abou...,does a rising oil price affect market share. w...
3,Reuters Authorities have halted oil export flo...,why is pipeline shut in southern iraq. who has...
4,AFP Tearaway world oil prices toppling records...,what are the oil prices?. what is the oil price
...,...,...
76870,RAMALLAH West Bank October No crowds of well w...,which u.s. state has been responsible for yass...
76871,For all the focus on foreign policy in this ca...,who is the president of iran today?. what is t...
76872,Security expert express concerns that cash mac...,what could happen if cash machines get viruses...
76873,Sixteen workers are critically hurt in a huge ...,where is oil refinery booleans located. where ...


In [18]:
queries= df_queries['Query'].tolist()

In [19]:
from sentence_transformers import  SentenceTransformer
model = SentenceTransformer('sentence-transformers/msmarco-MiniLM-L6-cos-v5')

In [20]:
#grt distinct corpus
queriesVectors= model.encode(queries)

In [21]:
queriesVectors.shape

(76875, 384)

In [22]:
with h5py.File('AG_News.hdf5', 'w') as f:
    f.create_dataset('test', data=queriesVectors)
    f.attrs['distance']= 'angular'

In [61]:
indicesList=[]
distances=[]
def search(queryVector, corpusVectors):
    from sklearn.neighbors import NearestNeighbors
    nbrs = NearestNeighbors(n_neighbors=100, algorithm='brute').fit(corpusVectors)
    _, indices = nbrs.kneighbors([queryVector])
    indicesList.append(indices)
    distances.append(_)

In [ ]:
import numpy as np

for q in queriesVectors:
    search(q, corpusVectors)
    
df_indices= pd.DataFrame(np.concatenate(indicesList), columns=[f"column_{i+1}" for i in range (100)])
df_distances= pd.DataFrame(np.concatenate(distances), columns=[f"column_{i+1}" for i in range (100)])

In [53]:
#df_distances.to_csv('distances.csv', index=False)
#df_indices.to_csv('indices.csv', index=False)

df_distances=pd.read_csv('distances.csv')
df_indices= pd.read_csv('neighbors.csv')

In [54]:
df_distances_arr= df_distances.values
df_indices_arr= df_indices.values

In [55]:
# with h5py.File('AG_News.hdf5', 'w') as f:
#     f.create_dataset('train', data=corpusVectors)
#     f.create_dataset('test', data=queriesVectors)
#     f.create_dataset('neighbors', data=df_indices_arr)
#     f.create_dataset('distances', data=df_distances_arr)
#     f.attrs['distance']= 'angular'

In [60]:
with h5py.File('AG_News.hdf5', 'r') as f:
    d=f['test']
    print(d[0].shape)


(384,)


In [62]:
l= "New york Accidents"
r= model.encode(l)
search(r, corpusVectors)

In [67]:
indicesList[0][0]

array([ 73170, 193170,  80340, 200340,  16231, 136231,  40350, 160350,
        33770, 153770,  47053, 167053, 127551,   7551, 134874,  14874,
        99119, 219119, 133900,  13900, 204119,  84119, 205150,  85150,
       153217,  33217, 168486,  48486, 136020,  16020, 204381,  84381,
       110730, 230730, 178668,  58668, 219110,  99110,  48662, 168662,
        17389, 137389, 201539,  81539,   4957, 124957, 131714,  11714,
       158047,  38047, 209623,  89623, 165897,  45897, 137452,  17452,
        84160, 204160, 176383,  56383, 168545,  48545, 188671,  68671,
       159163,  39163,  66290, 186290,  46788, 166788, 125122,   5122,
       103052, 223052, 201587,  81587,  71132, 191132,  62905, 182905,
         3322, 123322,  20931, 140931, 203616,  83616,  84377, 204377,
        56740, 176740,  36569, 156569, 223075, 103075, 194164,  74164,
        76079, 196079,  91960, 211960])